In [ ]:
import numpy as np
import keras
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.convolutional import Conv3D,MaxPooling3D
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras import Sequential
from keras.optimizers import SGD,RMSprop,Adam
from keras.callbacks import ModelCheckpoint
import sklearn
import pickle
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import keras.callbacks
from sklearn.model_selection import train_test_split
import os
import cv2

In [ ]:
XX = []
YY = []
j=0
base_path="../dataset/videos"
source_path=base_path
for child in os.listdir(source_path):
    print(child)
    sub_path = os.path.join(source_path, child)
    bsub_path = os.path.join(base_path, child)
    if os.path.isdir(sub_path):
        for data_file in os.listdir(sub_path):
            path=os.path.join(bsub_path,data_file)
            print(path)
            frames=[]
            cap=cv2.VideoCapture(path)
            fcount=cap.get(cv2.CAP_PROP_FRAME_COUNT)

            num_frames = 1000
            loop_limit = int(min(num_frames,fcount))
            print("Capturing {0} frames".format(num_frames))

            for i in range(0, loop_limit) :
                ret, frame = cap.read()
                gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
                resized=cv2.resize(gray, (224,224), interpolation = cv2.INTER_AREA)
                frames.append(resized)

            last_frame= frames[-1]
            for i in range(loop_limit, num_frames):
                frames.append(last_frame)
            XX.append(frames)
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

            cap.release()
            YY.append(child)
print(np.shape(XX))
print(np.shape(YY))


In [ ]:
X=np.reshape(XX,(27,224,224,1000,-1))
print(np.shape(X))
print(X)

In [ ]:
y=np.reshape(YY,(-1,1))
print(np.shape(y))

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
oh=OneHotEncoder()
y=oh.fit_transform(y)
lb=LabelBinarizer()
Y=lb.fit_transform(y)
print(Y)

In [ ]:
print(np.shape(Y))
print(X.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
model=Sequential()
model.add(Conv3D(128,(7,7,7),input_shape=(224,224,1000,1)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3,3,3)))


model.add(Conv3D(64,(3,3,3)))
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(3,3,3),padding='same'))
model.add(Dropout(0.5))


model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(25))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
checkpoint=ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(X_train,y_train,epochs=1000,callbacks=[checkpoint],validation_split=0.2)

In [ ]:

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))